# PREPARING DATA 4 VISUALIZATION IN TABLEAU

In [16]:
import os
import re

import pandas as pd
import numpy as np
from sklearn.externals import joblib
from sklearn.feature_selection import SelectKBest, f_classif, mutual_info_classif

In [26]:
def fit_transform_features(feature_selection, feat_test_values, target_test_values):
    """
    This function is to extract features in order to reduce the number of them.
    
    :param str feature_selection: Choose which stat use to reduce the features.
    :param np.Array feat_test_values: Numpy array with test features.
    :param np.Array target_test_values: Numpy array with test target.
    :return np.Array: Numpy array with the process done.
    """
    if feature_selection == 'f_classif':
        bests = SelectKBest(f_classif, k=20)
        feat_test_values = bests.fit_transform(feat_test_values, target_test_values)
    elif feature_selection == 'mutual':
        bests = SelectKBest(mutual_info_classif, k=20)
        feat_test_values = bests.fit_transform(feat_test_values, target_test_values)
    else:
        feat_test_values = feat_test_values

    return feat_test_values

In [5]:
def categorize_each_difference(num_list, df_):
    """
    This function categorize the shifted columns in Weak Bull o Bear (W. Bull, W. Bear),
    Bull or Bear and Strong Bull or Bear (S. Bull, S. Bear) depending on the value of the shifted column and
    his statistics (median, p25, p75) by ticker, year, month and sign.

    :param list num_list: List with days to categorize.
    :param pd.DataFrame df_: Dataframe to categorize.
    :return pd.DataFrame: Dataframe recalculated.
    """
    cols_to_keep = list(df_.columns)
    df_['year'], df_['month'] = df_['date'].dt.year, df_['date'].dt.month
    for num_ in num_list:
        df_.loc[df_['close_shifted_%i' % num_] >= 0, 'sign_%i' % num_] = 'Bull'
        df_.loc[df_['close_shifted_%i' % num_] < 0, 'sign_%i' % num_] = 'Bear'
        group = df_.groupby(['ticker', 'year', 'month', 'sign_%i' % num_])['close_shifted_%i' % num_].describe()
        group = group[['25%', '50%', '75%', 'std']].reset_index()
        group.rename({'std': 'std_%i' % num_,
                      '25%': '25_%i' % num_,
                      '50%': '50_%i' % num_,
                      '75%': '75_%i' % num_}, axis='columns', inplace=True)
        df_ = pd.merge(left=df_, right=group, on=['ticker', 'year', 'month', 'sign_%i' % num_], how='inner')

        df_.loc[(df_['sign_%i' % num_] == 'Bull') &
                (df_['close_shifted_%i' % num_] <= df_['50_%i' % num_]), 'cat_close_shifted_%i' % num_] = 'W. ' + df_[
            'sign_%i' % num_]
        df_.loc[(df_['sign_%i' % num_] == 'Bull') &
                (df_['close_shifted_%i' % num_] > df_['50_%i' % num_]) &
                (df_['close_shifted_%i' % num_] < df_['75_%i' % num_]), 'cat_close_shifted_%i' % num_] = df_['sign_%i' % num_]
        df_.loc[(df_['sign_%i' % num_] == 'Bull') &
                (df_['close_shifted_%i' % num_] >= df_['75_%i' % num_]), 'cat_close_shifted_%i' % num_] = 'S. ' + df_[
            'sign_%i' % num_]
        df_.loc[(df_['sign_%i' % num_] == 'Bear') &
                (df_['close_shifted_%i' % num_] >= df_['50_%i' % num_]), 'cat_close_shifted_%i' % num_] = 'W. ' + df_[
            'sign_%i' % num_]
        df_.loc[(df_['sign_%i' % num_] == 'Bear') &
                (df_['close_shifted_%i' % num_] < df_['50_%i' % num_]) &
                (df_['close_shifted_%i' % num_] > df_['25_%i' % num_]), 'cat_close_shifted_%i' % num_] = df_['sign_%i' % num_]
        df_.loc[(df_['sign_%i' % num_] == 'Bear') &
                (df_['close_shifted_%i' % num_] <= df_['25_%i' % num_]), 'cat_close_shifted_%i' % num_] = 'S. ' + df_[
            'sign_%i' % num_]

        cols_to_keep.extend(['cat_close_shifted_%i' % num_, '50_%i' % num_, '75_%i' % num_, '25_%i' % num_])
    return df_[cols_to_keep]

In [6]:
def add_shifts(df_, col_to_shift, new_col, shift):
    """
    This function add shifted columns to data by ticker.

    :param pd.DataFrame df_: Dataframe with financial data.
    :param str col_to_shift: Column over to create the shift.
    :param str new_col: Name of the shifted column.
    :param int shift: Days to use as shift.
    :return pd.DataFrame: Dataframe with the shift added.
    """

    for id_ in df_['ticker'].unique():
        df_by_id = df_[df_['ticker'] == id_]
        df_.loc[df_['ticker'] == id_, new_col] = -df_by_id[col_to_shift] + df_by_id[col_to_shift].shift(shift)

    return df_

In [7]:
def interpolate_nan_values(df_, to_interpolate):
    """
    Interpolate and extrapolate nan values for numerical columns.

    :param pd.DataFrame df_: Dataframe with financial data with NaN values.
    :param list to_interpolate: List with columns to interpolate.
    :return pd.DataFrame: Dataframe with financial data without NaN values.
    """

    list_df = []
    for tick in df_['ticker'].unique():
        df_by_ticker = df_[df_['ticker'] == tick]
        for col in to_interpolate:
            df_by_ticker[col] = df_by_ticker[col].interpolate(method='linear', limit_direction='both')
        list_df.append(df_by_ticker)
    return pd.concat(list_df)

In [8]:
def load_and_transform():
    """
    Load and transform categorical and financial data to use in models.

    :return pd.DataFrame: Dataframe to use in models.
    """
    df_categorical = pd.read_csv('../data/db_bsm_categorical.csv')
    df_financial = pd.read_csv('../data/db_bsm_financial.csv')
    df_financial.replace(0, np.NaN, inplace=True)
    df_financial_not_nan = interpolate_nan_values(df_financial, ['close', 'volume'])
    df_financial_not_nan = df_financial_not_nan.sort_values(['ticker', 'date'], ascending=[True, False])
    for num_ in [3, 5, 7, 14, 21]:
        df_financial_not_nan = add_shifts(df_financial_not_nan, 'close', 'close_shifted_%i' % num_, num_)

    df_financial_not_nan.dropna(subset=['close_shifted_21'], inplace=True)
    df_fin_not_nan = interpolate_nan_values(df_financial_not_nan, list(df_financial_not_nan.select_dtypes(float)))
    df_fin_not_nan['date'] = pd.to_datetime(df_fin_not_nan['date'])
    df_fin_not_nan = df_fin_not_nan[df_fin_not_nan['date'].dt.year >= 2019]
    df_final = categorize_each_difference([3, 5, 7, 14, 21], df_fin_not_nan)
    df_categorical = df_categorical.dropna()
    df_categorical = df_categorical.drop_duplicates(subset=['ticker'], keep='first')
    df_final = pd.merge(left=df_final, right=df_categorical, how='inner', on='ticker')
    df_final.replace(0, np.NaN, inplace=True)
    df_final[df_final.select_dtypes(float).columns] = df_final.select_dtypes(float).astype('float32')
    df_final.replace([np.inf, -np.inf], np.NaN, inplace=True)
    df_final = df_final.dropna()
    df_final[df_final.select_dtypes('float32').columns] = df_final.select_dtypes('float32').astype(float)
    return df_final

In [9]:
def get_non_n_cols(df_, n):
    """
    Get the columns that his time window is less than n days.

    :param pd.DataFrame df_: Dataframe with financial data.
    :param int n: n days to get columns with.
    :return list: List with the name of the columns.
    """

    return [elem for elem in df_.columns if
            (re.search(r'\d+$', elem) is not None) and (int(elem[-2:].strip().strip('_')) < n)]

In [10]:
def get_unwanted_cols(df_):
    """
    This functions gives columns to drop from df_.

    :param pd.DataFrame df_: Dataframe to calc unwanted columns.
    :return list: Name of the unwanted columns from df_.
    """
    return [elem for elem in df_.columns if
            elem.startswith('close_shifted') or
            elem.startswith('cat_close_shifted') or
            elem.startswith('std') or
            elem.startswith('25') or 
            elem.startswith('50') or 
            elem.startswith('75')]

In [11]:
%time df_2_proc = load_and_transform()

C:\Users\esteb\Anaconda3\lib\site-packages\ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


Wall time: 5min 23s


In [21]:
cols_to_keep = ['close','date', 'ticker', 'close_shifted_3', 'close_shifted_5', 'close_shifted_7', 'close_shifted_14', 'close_shifted_21', 'cat_close_shifted_3',
                '50_3', '75_3', '25_3', 'cat_close_shifted_5', '50_5', '75_5', '25_5', 'cat_close_shifted_7', '50_7', '75_7', '25_7', 'cat_close_shifted_14', '50_14',
                '75_14', '25_14', 'cat_close_shifted_21', '50_21', '75_21', '25_21', 'company', 'sector_gics', 'stock_index', 'country', 'pred_3', 'pred_5', 'pred_7', 
                'pred_14', 'pred_21']

In [22]:
list_preds = []
list_preds_df = []
feature_selection = 'mutual'
for sector_ in list(df_2_proc['sector_gics'].unique()):
    for num_ in [3, 5, 7, 14, 21]:
        list_to_drop = get_non_n_cols(df_2_proc, num_)
        df_ = df_2_proc[df_2_proc['sector_gics'] == sector_]
        df_ = df_.drop(list_to_drop, axis='columns')
        if (feature_selection == 'f_classif') or (feature_selection == 'mutual'):
            path = '../models_%s/%s/%i' % (feature_selection, sector_, num_)
        else:
            path = '../models/%s/%i' % (sector_, num_)
        path = os.path.relpath(path)
        list_files = os.listdir(path)
        for file in list_files:
            if file.endswith('.mdl'):
                model_path = os.path.join(path, file)
        mdl = joblib.load(model_path)
        y_test = df_['cat_close_shifted_%d' % num_]
        X_test = df_[df_.select_dtypes(float).columns]
        list_to_drop_2 = get_unwanted_cols(X_test)
        X_test = X_test.drop(list_to_drop_2, axis='columns')
        X_test_values = X_test.values
        y_test_values = y_test.values
        X_test_values = fit_transform_features(feature_selection, X_test_values, y_test_values)
        y_pred = mdl.predict(X_test_values)
        y_pred = y_pred.tolist()
        y_pred = pd.DataFrame({'pred_%i' % num_: y_pred})
        list_preds.append(y_pred)
    df_preds = pd.concat(list_preds, axis='columns')
    list_preds = []
    df_filtered = df_2_proc[df_2_proc['sector_gics'] == sector_].reset_index(drop=True)
    df_tot = pd.concat([df_filtered, df_preds], axis='columns')
    df_tot = df_tot[cols_to_keep]
    list_preds_df.append(df_tot)
df_fin = pd.concat(list_preds_df, axis='index')

In [23]:
for num_ in [3, 5, 7, 14, 21]:
    df_fin['ccs_%i_abs' % num_] = df_fin['cat_close_shifted_%i' % num_].apply(lambda _: _[-4:])
    df_fin['ccs_%i_pred' % num_] = df_fin['pred_%i' % num_].apply(lambda _: _[-4:])

In [24]:
if (feature_selection == 'f_classif') or (feature_selection == 'mutual'):
    path_to_save = '../data/4vis_%s.csv' % feature_selection
else:
    path_to_save = '../data/4vis.csv'
df_fin.to_csv(path_to_save, index=False, sep=';', decimal=',')

In [25]:
df_fin.shape

(2348, 47)